In [1]:
!pip install -q -U pyomo
!apt-get install -y -qq glpk-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.4 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123589 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

In [2]:
num_shifts = 3  # Number of shifts in a day
time_slots_per_shift = 8
time_slots = range(num_shifts * time_slots_per_shift)  # 24 hours in a day
call_volume_forecast = [50, 40, 45, 30, 25, 20, 15, 10,
                        20, 30, 40, 50, 60, 70, 80, 90,
                        0, 10, 1, 1, 40, 50, 60, 70]  # Example forecast
cost_per_agent = 100  # $20 per agent per shift
max_agents_per_shift = 100  # Maximum number of agents allowed per shift
total_budget = 1500
income_per_call = 30  # Income generated per call answered


In [3]:
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Define variables
model.x = Var(range(num_shifts), within=NonNegativeIntegers)  # Number of agents per shift

# Objective function: Maximize profit (income - cost)
def profit(model):
    total_income = sum(call_volume_forecast[i] * income_per_call for i in time_slots)
    total_cost = sum(model.x[j] * cost_per_agent for j in range(num_shifts))
    return total_income - total_cost

model.obj = Objective(rule=profit, sense=maximize)

# Constraints
def service_level_constraint(model, i):
  shift = i // time_slots_per_shift
  # Each agent handles one time slot, so the number of agents should be at least the call volume for that time slot
  # return model.x[shift] >= call_volume_forecast[i]
  return model.x[shift] >= call_volume_forecast[i] / income_per_call


def max_agents_constraint(model, j):
    return model.x[j] <= max_agents_per_shift

def budget_constraint(model):
    # return sum(list(model.x) * cost_per_agent) <= total_budget
    return sum(model.x[j] * cost_per_agent for j in range(num_shifts)) <= total_budget

model.service_level = Constraint(time_slots, rule=service_level_constraint)
model.max_agents = Constraint(range(num_shifts), rule=max_agents_constraint)
model.budget = Constraint(rule=budget_constraint)

# Solve the model using GLPK
solver = SolverFactory('glpk')
result = solver.solve(model, tee=False)

# Check solver status and termination condition
if result.solver.status == SolverStatus.ok and result.solver.termination_condition == TerminationCondition.optimal:
    print("\nOptimal solution found.")
    # Print results
    print("\nOptimal Number of Agents per Shift:")
    for j in range(num_shifts):
        print(f"Shift {j+1}: {model.x[j].value} agents")

    total_cost = sum(cost_per_agent * model.x[j].value for j in range(num_shifts))
    total_income = sum(call_volume_forecast[i] * income_per_call for i in time_slots)
    profit = total_income - total_cost
    print(f"\nTotal Staffing Cost: ${total_cost:.2f}")
    print(f"Total Income: ${total_income:.2f}")
    print(f"Profit: ${profit:.2f}")

    # Check constraints
    service_level_satisfied = all(model.x[i // time_slots_per_shift].value * time_slots_per_shift >= call_volume_forecast[i] for i in time_slots)
    budget_satisfied = total_cost <= total_budget
    max_agents_satisfied = all(model.x[j].value <= max_agents_per_shift for j in range(num_shifts))

    print("\nConstraint Satisfaction:")
    print(f"Service Level Constraints Satisfied: {service_level_satisfied}")
    print(f"Budget Constraint Satisfied: {budget_satisfied}")
    print(f"Max Agents per Shift Constraints Satisfied: {max_agents_satisfied}")

    print("\nSolver Status and Termination Condition:")
    print(f"Solver Status: {result.solver.status}")
    print(f"Termination Condition: {result.solver.termination_condition}")
elif result.solver.termination_condition == TerminationCondition.infeasible:
    print("\nNo feasible solution found.")
else:
    print("\nSolver encountered an issue. Status: ", result.solver.status)



Optimal solution found.

Optimal Number of Agents per Shift:
Shift 1: 2.0 agents
Shift 2: 3.0 agents
Shift 3: 3.0 agents

Total Staffing Cost: $800.00
Total Income: $27210.00
Profit: $26410.00

Constraint Satisfaction:
Service Level Constraints Satisfied: False
Budget Constraint Satisfied: True
Max Agents per Shift Constraints Satisfied: True

Solver Status and Termination Condition:
Solver Status: ok
Termination Condition: optimal


Optimal Number of Agents per Shift:
- Shift 1: 5.0 agents
- Shift 2: 9.0 agents
- Shift 3: 7.0 agents

- Total Staffing Cost: $1050.00

- Total Income: $9070.00

- Profit: $8020.00


Optimal Number of Agents per Shift:
- Shift 1: 2.0 agents
- Shift 2: 3.0 agents
- Shift 3: 3.0 agents

- Total Staffing Cost: $800.00

- Total Income: $27210.00

- Profit: $26410.00


In [15]:
type(model.x)

In [18]:
print(model.x)

x


In [23]:
sum(list(model.x)*2)

6

In [27]:
print(sum(model.x[j] * 2 for j in range(num_shifts)))

2*x[0] + 2*x[1] + 2*x[2]
